
### Dependencies and Setup


In [ ]:
knitr::opts_chunk$set(echo = TRUE)

#install.packages("ggplot2")
#install.packages("dplyr")
#install.packages("readxl")
#install.packages("latex2exp")
# 
library(ggplot2)
library(dplyr)
library(readxl)
library(latex2exp)


# Simulación de un proceso AR(1)

Definamos la serie de tiempo, el proceso AR(1) es: 

\begin{equation}
  X_t = a_0 + a_1 X_{t-1} + U_t
\end{equation}

# Parametros:


In [ ]:
a0 <- 5; a1 <- 0.9; X_0 <- (a0/(1 - a1)); T <- 1000



# Definimos un data frame para almacenar el proceso, agregamos una columna para el tiempo



In [ ]:
X_t <- data.frame(Tiempo = c(0:T))

View(X_t)


#  Parte estocastica de la serie de tiempo:



In [ ]:
set.seed(12345)

# Agregamos un término estocástico al data frame

X_t$U_t <- rnorm(T+1, mean = 0, sd = 4)

View(X_t)


# GRÁFICA



In [ ]:
ggplot(data = X_t, aes(x = Tiempo, y = U_t)) + 
  geom_line(size = 0.5, color = "darkblue") +
  #theme_bw() + 
  xlab("Tiempo") + 
  ylab(TeX("$u_t$")) + 
  theme(plot.title = element_text(size = 11, face = "bold", hjust = 0)) + 
  theme(plot.subtitle = element_text(size = 10, hjust = 0)) + 
  theme(plot.caption = element_text(size = 10, hjust = 0)) +
  theme(plot.margin = unit(c(1,1,1,1), "cm")) +
  labs(
    title = "Proceso 'puramente' aleatorio",
    subtitle = "Distribución Normal, media = 0, desviación estándar = 4",
    caption = "Fuente: Elaboración propia."
  )


# Trayectoria del proceso AR(1) y de la solucion del mismo:
\begin{equation}
  X_t = \frac{a_0}{1 - a_1} + \sum a_1^j U_{t-j}
\end{equation}


In [ ]:
# Agregamos columnas con NA's para un proceso teorico y uno real
X_t$X_t <- NA
X_t$XR_t <- NA

# La serie teórica inicia en un valor inicial X_0
X_t$X_t[1] <- X_0

# La serie real inicia en un valor inicial X_0
X_t$XR_t[1] <- X_0

# Agregamos una columna para la función de Autocorrelación teórica:
X_t$rho <-NA


# Construcción de dos procesos: uno 'real' y uno 'estimado' o teórico y autocorrelación



In [ ]:
for (i in 2:(T + 1)) {
  # Real:
  X_t$XR_t[i] = a0 + a1*X_t$XR_t[i-1] + X_t$U_t[i-1]
  
  # Teórico:
  X_t$X_t[i] = X_t$X_t[i-1] + (a1^(i-1))*X_t$U_t[i-1]
  
  # Autocorrelación:
  X_t$rho[i-1] = a1^(i-1)
}


# Graficas:



In [ ]:
ggplot(data = X_t, aes(x = Tiempo, y = X_t)) + 
  geom_line(size = 0.5, color = "darkblue") +
  #theme_bw() + 
  xlab("Tiempo") + 
  ylab(TeX("$X_t$")) + 
  theme(plot.title = element_text(size = 11, face = "bold", hjust = 0)) + 
  theme(plot.subtitle = element_text(size = 10, hjust = 0)) + 
  theme(plot.caption = element_text(size = 10, hjust = 0)) +
  theme(plot.margin = unit(c(1,1,1,1), "cm")) +
  labs(
    title = "Comportamiento del Proceso Teórico",
    subtitle = "Con un error con Distribución Normal (media = 0, desviación estándar = 4)",
    caption = "Fuente: Elaboración propia."
  )

ggsave("G_AR_1_Teo.png", width = 20, height = 10, units = "cm")


In [ ]:
ggplot(data = X_t, aes(x = Tiempo, y = XR_t)) + 
  geom_line(size = 0.5, color = "darkred") +
  #theme_bw() + 
  xlab("Tiempo") + 
  ylab(TeX("$X_t$")) + 
  theme(plot.title = element_text(size = 11, face = "bold", hjust = 0)) + 
  theme(plot.subtitle = element_text(size = 10, hjust = 0)) + 
  theme(plot.caption = element_text(size = 10, hjust = 0)) +
  theme(plot.margin = unit(c(1,1,1,1), "cm")) +
  labs(
    title = "Comportamiento del Proceso Real ('Estimado')",
    subtitle = "Con un error con Distribución Normal (media = 0, desviación estándar = 4)",
    caption = "Fuente: Elaboración propia."
  )

ggsave("G_AR_1_Real.png", width = 20, height = 10, units = "cm")


In [ ]:
ggplot(data = X_t, aes(x = Tiempo)) +
  geom_line(aes(y = XR_t), size = 0.5, color = "darkred") +
  geom_line(aes(y = X_t), size = 0.5, color = "darkblue") +
  #theme_bw() + 
  xlab("Tiempo") + 
  ylab(TeX("$X_t$")) + 
  theme(plot.title = element_text(size = 11, face = "bold", hjust = 0)) + 
  theme(plot.subtitle = element_text(size = 10, hjust = 0)) + 
  theme(plot.caption = element_text(size = 10, hjust = 0)) +
  theme(plot.margin = unit(c(1,1,1,1), "cm")) +
  labs(
    title = "Comportamiento de los Procesos Real y Teórico",
    subtitle = "Con un error con Distribución Normal (media = 0, desviación estándar = 4)",
    caption = "Fuente: Elaboración propia."
  )

ggsave("G_AR_1_Comb.png", width = 20, height = 10, units = "cm")


In [ ]:
acf(X_t$XR_t, lag.max = 30, col = "blue", 
    ylab = "Autocorrelacion",
    xlab="Rezagos", 
    main="Funcion de Autocorrelacion Real")


In [ ]:
barplot(X_t$rho[1:30], names.arg = c(1:30), col = "blue", border="blue", density = c(10,20), 
        ylab = "Autocorrelacion", 
        xlab="Rezagos", 
        main="Funcion de Autocorrelacion Teórica")
